In [13]:
import pandas as pd
import random
import csv
import numpy as np
from pandas.api.types import is_string_dtype
import random
from datetime import datetime, timedelta
from collections import defaultdict
import os

In [14]:
from google.colab import drive
drive.mount('/content/drive')

os.chdir("/content/drive/MyDrive/test_data")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [54]:
equipment_df = pd.read_csv("Equipment.csv")
stage_df = pd.read_csv("Stage.csv")
festival_df = pd.read_csv("Festival.csv")
subgenre_df = pd.read_csv("sub_genre.csv")
staff_df = pd.read_csv("Staff.csv")
performance_type_df = pd.read_csv("Performance_type.csv")
artist_df = pd.read_csv("Artist.csv")
visitor_df = pd.read_csv("Visitor.csv")
ticket_type_df = pd.read_csv("Ticket_type.csv")
payment_method_df = pd.read_csv("payment_method.csv")
ticket_type_df = pd.read_csv("Ticket_type.csv")

In [21]:
event_df = pd.read_csv("generated/event.csv")

In [ ]:
performer_df = pd.read_csv("generated/Performer.csv")
artist_has_performer_df = pd.read_csv("generated/artist_has_performer.csv")
ticket_df = pd.read_csv("generated/Ticket.csv")
performance_df = pd.read_csv("generated/Performance.csv")

In [18]:
#equipment_has_stage
equipment_has_stage = {
    "Equipment_Name":[],
    "Stage_Stage_id":[],
    "Quantity":[]
}
equip_name = equipment_df["Name"].tolist()
stage_id = stage_df["Stage_id"].tolist()

for stage in stage_id:
  for equip in equip_name:
    q = random.randint(2,15)
    equipment_has_stage["Stage_Stage_id"].append(stage)
    equipment_has_stage["Equipment_Name"].append(equip)
    equipment_has_stage["Quantity"].append(q)

pd.DataFrame(equipment_has_stage).to_csv("generated/equipment_has_stage.csv", index=False, quoting=csv.QUOTE_NONNUMERIC, quotechar='"')

In [64]:
#performer_has_subgenre
performer_has_subgenre = {
    "Performer_Performer_id":[],
    "Subgenre_Subgenre_id":[]
}

perf_id = performer_df["Performer_id"].tolist()
sub_id = subgenre_df["Subgenre_id"].tolist()

for performer in perf_id:
  for sub in sub_id:
    if random.random() < 0.3:
      performer_has_subgenre["Performer_Performer_id"].append(performer)
      performer_has_subgenre["Subgenre_Subgenre_id"].append(sub)

pd.DataFrame(performer_has_subgenre).to_csv("generated/performer_has_subgenre.csv", index = False, quoting=csv.QUOTE_NONNUMERIC, quotechar='"')

In [20]:
#events
event_records = []
used_slots = {}  # key = (stage_id, date)

for _, fest in festival_df.iterrows():
    fest_days = pd.date_range(start=fest["StartDate"], end=fest["EndDate"])
    for day in fest_days:
        stage_id = random.choice(stage_df["Stage_id"].tolist())
        if (stage_id, day.date()) in used_slots:
            continue  # skip overlapping
        used_slots[(stage_id, day.date())] = True
        event_records.append({
            "Event_id": len(event_records)+1,
            "Date": day.date(),
            "Festival_Festival_id": fest["Festival_id"],
            "Stage_Stage_id": stage_id
        })

event_df = pd.DataFrame(event_records)
event_df.to_csv("generated/event.csv", index=False, quoting=csv.QUOTE_NONNUMERIC, quotechar='"')

In [71]:
#event_has_staff
staff_roles = staff_df.groupby("Role_Νame")["Staff_id"].apply(list).to_dict()
stage_capacity = stage_df.set_index("Stage_id")["Capacity"].to_dict()

event_has_staff = {
    "Event_Event_id": [],
    "Staff_Staff_id": []
}

# Χάρτης διαθεσιμότητας προσωπικού: {Staff_id: [ημερομηνίες]}
staff_availability = defaultdict(set)

for _, event in event_df.iterrows():
    event_id = event["Event_id"]
    event_date = pd.to_datetime(event["Date"]).date()
    stage_id = event["Stage_Stage_id"]
    cap = stage_capacity[stage_id]

    required = {
        "Ασφάλεια": max(1, int(cap * 0.05)),
        "Βοηθητικό": max(1, int(cap * 0.02)),
        "Τεχνικό": random.randint(2, 4)
    }

    for role, count in required.items():
        available_staff = [
            sid for sid in staff_roles.get(role, [])
            if event_date not in staff_availability[sid]
        ]
        selected = random.sample(available_staff, min(count, len(available_staff)))

        for sid in selected:
            event_has_staff["Event_Event_id"].append(event_id)
            event_has_staff["Staff_Staff_id"].append(sid)
            staff_availability[sid].add(event_date)  # Mark as busy that day

pd.DataFrame(event_has_staff).to_csv("generated/event_has_staff.csv", index=False, quoting=csv.QUOTE_NONNUMERIC, quotechar='"')


KeyError: 'Role_Νame'

In [70]:
#performance


# βοηθοί
performance = {
    "Performance_id": [],
    "StartTime": [],
    "DurationINT": [],
    "BreakDurationINT": [],
    "Event_Event_id": [],
    "Performer_Performer_id": [],
    "Performance_type_Type": []
}

performance_id = 1

# 1. κρατάμε για κάθε performer πότε εμφανίστηκε (για overlap)
performer_schedule = defaultdict(list)  # {performer_id: [ (start, end) ]}
# 2. για κάθε performer: έτη συμμετοχής
performer_years = defaultdict(set)  # {performer_id: set(έτη)}
# 3. για κάθε artist: ποια groups/performers ανήκει
artist_performer_map = defaultdict(list)
for _, row in artist_has_performer_df.iterrows():
    artist_performer_map[row["Artist_Artist_id"]].append(row["Performer_Performer_id"])

# 4. helper για να ελέγξουμε overlap
def is_available(p_id, new_start, new_end):
    for s, e in performer_schedule[p_id]:
        if not (new_end <= s or new_start >= e):
            return False
    return True

# για κάθε event
for _, event in event_df.iterrows():
    event_id = event["Event_id"]
    event_date = pd.to_datetime(event["Date"]).date()
    festival_year = festival_df[festival_df["Festival_id"] == event["Festival_Festival_id"]]["Year"].values[0]

    # ξεκινάμε χρονικά από μια βάση ώρας
    current_time = datetime.combine(event_date, datetime.strptime("10:00", "%H:%M").time())

    max_performers = random.randint(2, 4)
    chosen_performers = []

    attempts = 0
    while len(chosen_performers) < max_performers and attempts < 100:
        attempts += 1
        candidate = random.choice(performer_df["Performer_id"].tolist())

        # 1. ελεγξε "μέχρι 3 συνεχή έτη"
        years = performer_years[candidate]
        if len(years) >= 3:
            years_sorted = sorted(years)
            for i in range(len(years_sorted) - 2):
                if years_sorted[i+2] - years_sorted[i] == 2:  # 3 συνεχόμενα έτη
                    continue  # έχει ήδη 3 συνεχόμενες συμμετοχές

        # 2. ελεγξε "διαθεσιμότητα" performer (και artists του αν είναι group)
        duration = random.randint(30, 180)
        break_time = random.randint(5, 30)
        start_time = current_time
        end_time = current_time + timedelta(minutes=duration)

        if not is_available(candidate, start_time, end_time):
            continue

        # αν είναι group, ελεγξε και artists
        artists_in_group = artist_has_performer_df[
            artist_has_performer_df["Performer_Performer_id"] == candidate
        ]["Artist_Artist_id"].tolist()

        valid = True
        for artist_id in artists_in_group:
            for performer_of_artist in artist_performer_map[artist_id]:
                if not is_available(performer_of_artist, start_time, end_time):
                    valid = False
                    break
            if not valid:
                break
        if not valid:
            continue

        # passed all checks
        chosen_performers.append(candidate)
        performance["Performance_id"].append(performance_id)
        performance["StartTime"].append(start_time)
        performance["DurationINT"].append(duration)
        performance["BreakDurationINT"].append(break_time)
        performance["Event_Event_id"].append(event_id)
        performance["Performer_Performer_id"].append(candidate)
        performance["Performance_type_Type"].append(random.choice(performance_type_df["Type"].tolist()))

        # ενημέρωσε χάρτες
        performer_schedule[candidate].append((start_time, end_time))
        performer_years[candidate].add(festival_year)
        for artist_id in artists_in_group:
            for pid in artist_performer_map[artist_id]:
                performer_schedule[pid].append((start_time, end_time))

        performance_id += 1
        current_time = end_time + timedelta(minutes=break_time)

pd.DataFrame(performance).to_csv("generated/performance.csv", index=False, quoting=csv.QUOTE_NONNUMERIC, quotechar='"')


In [35]:
#performer
#artist_has_performer

# Δημιουργία φακέλου αν δεν υπάρχει
os.makedirs("generated", exist_ok=True)

# Ανακάτεμα των καλλιτεχνών
artist_df = artist_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Παράμετροι
num_artists = len(artist_df)
solo_percentage = 0.7  # 70% solo

# Υπολογισμός αριθμού καλλιτεχνών που θα είναι solo
num_solo_artists = int(num_artists * solo_percentage)
num_group_artists = num_artists - num_solo_artists

# Επιλογή ποιοι καλλιτέχνες θα είναι solo
solo_artist_indices = set(random.sample(range(num_artists), num_solo_artists))
group_artist_indices = set(range(num_artists)) - solo_artist_indices

solo_artists = artist_df.iloc[list(solo_artist_indices)].copy().reset_index(drop=True)
group_artists = artist_df.iloc[list(group_artist_indices)].copy().reset_index(drop=True)

# Τυχαίος διαχωρισμός group καλλιτεχνών σε groups 2-7 μελών
group_sizes = []
remaining = len(group_artists)

while remaining > 0:
    size = random.randint(2, 7)
    if size > remaining:
        size = remaining
    group_sizes.append(size)
    remaining -= size

# Λίστα λέξεων για ονόματα συγκροτημάτων
word_list = [
    "Echo", "Nova", "Electric", "Fusion", "Sonic", "Crimson", "Velvet", "Nebula",
    "Quantum", "Rogue", "Lunar", "Vibe", "Zenith", "Orbit", "Bliss", "Jungle", "Tiger", "Boom", "Hell"
]

performers = []
artist_has_performer = []
performer_id = 1

# Solo artists -> κάθε ένας είναι Performer
for _, row in solo_artists.iterrows():
    stage_name = row["Name"].replace(' ', '').lower()
    performers.append({
        "Performer_id": performer_id,
        "Name": row["Name"],
        "FormationDate": pd.to_datetime("2015-01-01") + pd.to_timedelta(random.randint(0, 3000), unit="D"),
        "Website": f"http://{stage_name}.com",
        "InstagramProfile": f"https://instagram.com/{stage_name}"
    })
    artist_has_performer.append({
        "Artist_Artist_id": row["Artist_id"],
        "Performer_Performer_id": performer_id
    })
    performer_id += 1

# Group artists -> Φτιάχνουμε τα groups
idx = 0
for size in group_sizes:
    members = group_artists.iloc[idx:idx+size]
    group_name = " ".join(random.sample(word_list, k=random.choice([1, 2])))
    formation_date = pd.to_datetime("2010-01-01") + pd.to_timedelta(random.randint(0, 4000), unit="D")

    performers.append({
        "Performer_id": performer_id,
        "Name": group_name,
        "FormationDate": formation_date,
        "Website": f"http://{group_name.replace(' ', '').lower()}.com",
        "InstagramProfile": f"https://instagram.com/{group_name.replace(' ', '').lower()}"
    })

    for _, member in members.iterrows():
        artist_has_performer.append({
            "Artist_Artist_id": member["Artist_id"],
            "Performer_Performer_id": performer_id
        })

    performer_id += 1
    idx += size

# Αποθήκευση στα CSV αρχεία
pd.DataFrame(performers).to_csv("generated/performer.csv", index=False, quoting=csv.QUOTE_NONNUMERIC, quotechar='"')
pd.DataFrame(artist_has_performer).to_csv("generated/artist_has_performer.csv", index=False, quoting=csv.QUOTE_NONNUMERIC, quotechar='"')

print("Τα αρχεία δημιουργήθηκαν επιτυχώς!")


Τα αρχεία δημιουργήθηκαν επιτυχώς!


In [56]:
#ticket

# Λίστα με τους διαθέσιμους τύπους εισιτηρίων
ticket_types = ["VIP", "Early Bird", "Regular", "Backstage", "Group"]

# Mapping για stage capacity
stage_capacity = dict(zip(stage_df["Stage_id"], stage_df["MaxCapacity"]))

# Για να αποφεύγουμε διπλό εισιτήριο του ίδιου visitor στο ίδιο event
visitor_event_tickets = set()

today = pd.to_datetime(datetime.today().date())
ticket_data = []
ticket_id = 1

for _, event in event_df.iterrows():
    event_id = event["Event_id"]
    stage_id = event["Stage_Stage_id"]
    event_date = pd.to_datetime(event["Date"])
    max_capacity = stage_capacity.get(stage_id, 100)  # fallback αν λείπει κάποιο capacity

    # Πόσα εισιτήρια θα δημιουργήσουμε
    num_tickets = max_capacity

    # Κατανομή εισιτηρίων
    num_vip = int(num_tickets * 0.10)          # 10%
    num_early_bird = int(num_tickets * 0.15)   # 15%
    num_backstage = int(num_tickets * 0.05)    # 5%
    num_group = int(num_tickets * 0.05)        # 5%
    num_regular = num_tickets - num_vip - num_early_bird - num_backstage - num_group  # 65%

    # Βάση τιμής ανά event
    base_price = random.randint(40, 120)

    # Τιμές για κάθε τύπο εισιτηρίου
    price_multipliers = {
        "VIP": 2.5,
        "Early Bird": 0.8,
        "Regular": 1.0,
        "Backstage": 3.0,
        "Group": 0.7
    }

    # Ανακατεύουμε visitors
    visitors = visitor_df.sample(frac=1).reset_index(drop=True)
    visitor_index = 0

    event_happened = event_date < today

    # Δημιουργία λίστας με τους τύπους εισιτηρίων για κάθε θέση
    ticket_type_distribution = (["VIP"] * num_vip +
                              ["Early Bird"] * num_early_bird +
                              ["Backstage"] * num_backstage +
                              ["Group"] * num_group +
                              ["Regular"] * num_regular)
    random.shuffle(ticket_type_distribution)

    for i in range(num_tickets):
        active = 0
        payment = None
        # Είδος εισιτηρίου
        current_type = ticket_type_distribution[i]
        price = int(base_price * price_multipliers[current_type])

        # Ημερομηνία αγοράς
        days_before_event = (event_date - today).days if event_happened else 0

        if event_happened:
            if current_type == "Early Bird":
                # Early Bird εισιτήρια αγοράζονται 30-90 μέρες πριν
                min_days = min(90, days_before_event)
                max_days = max(30, min_days)
                purchase_date = event_date - timedelta(days=random.randint(30, max_days))
            else:
                # Άλλα εισιτήρια αγοράζονται 1-30 μέρες πριν
                min_days = min(30, days_before_event)
                max_days = max(1, min_days)
                purchase_date = event_date - timedelta(days=random.randint(1, max_days))
        else:
            # Για μελλοντικά events
            purchase_date = today - timedelta(days=random.randint(0, 30))

        # Διόρθωση αν η ημερομηνία είναι στο μέλλον
        if purchase_date > today:
            purchase_date = today

        # Αρχικοποίηση visitor_id ως None
        visitor_id = None

        if event_happened and random.random() < 0.8:
            # 80% των εισιτηρίων συνδέονται με visitor
            while visitor_index < len(visitors):
                potential_visitor_id = int(visitors.iloc[visitor_index]["Visitor_id"])
                visitor_index += 1
                if (potential_visitor_id, event_id) not in visitor_event_tickets:
                    visitor_id = potential_visitor_id
                    visitor_event_tickets.add((visitor_id, event_id))
                    break
            active = 1
            payment = random.choice(payment_method_df["Method"].tolist())

        # Καταχώρηση του εισιτηρίου
        ticket_data.append({
            "Ticket_id": ticket_id,
            "PurchaseDate": purchase_date,
            "Price": price,
            "EANCode": f"{random.randint(1000000000000, 9999999999999)}",
            "IsActive": active,
            "Event_Event_id": event_id,
            "Visitor_Visitor_id": visitor_id,
            "Ticket_type_Type": current_type,
            "Payment_method_Method": payment,
            "Seller_Seller_id": None
        })

        ticket_id += 1

# Δημιουργούμε το DataFrame
ticket_df = pd.DataFrame(ticket_data)

# Μετατρέπουμε τη στήλη Visitor_Visitor_id σε Int64 (επιτρέπει NULL values)
ticket_df["Visitor_Visitor_id"] = pd.to_numeric(ticket_df["Visitor_Visitor_id"], errors='coerce').astype('Int64')

# Αποθήκευση σε CSV
ticket_df.to_csv("generated/ticket.csv", index=False, quoting=csv.QUOTE_NONNUMERIC, quotechar='"')

print("Το αρχείο ticket.csv δημιουργήθηκε επιτυχώς!")

Το αρχείο ticket.csv δημιουργήθηκε επιτυχώς!


In [59]:
# buyer has ticket

ticket_df = pd.read_csv("generated/ticket.csv")
buyer_has_ticket = []

# 1. Βρες μελλοντικά events (που δεν έχουν γίνει ακόμα)
future_events = event_df[pd.to_datetime(event_df["Date"]) > datetime.now()]
future_event_ids = future_events["Event_id"].tolist()

# 2. Βρες events όπου ΟΛΑ τα εισιτήρια έχουν Visitor_id (έχουν αγοραστεί)
sold_out_events = ticket_df[
    (ticket_df["Event_Event_id"].isin(future_event_ids)) &
    (ticket_df["Visitor_Visitor_id"].notna())
].groupby("Event_Event_id").filter(
    lambda x: x["Visitor_Visitor_id"].count() == len(x)
)["Event_Event_id"].unique().tolist()

# 3. Για κάθε sold-out event
for event_id in sold_out_events:
    # Τύποι εισιτηρίων που υπάρχουν για αυτό το event
    event_ticket_types = ticket_df[ticket_df["Event_Event_id"] == event_id]["Ticket_type_Type"].unique()

    # Επιλογή τυχαίων visitors (20-30% του συνόλου)
    num_visitors = random.randint(
        int(len(visitor_df) * 0.2),
        int(len(visitor_df) * 0.3)
    )
    interested_visitors = random.sample(visitor_df["Visitor_id"].tolist(), num_visitors)

    # Για κάθε visitor
    for visitor_id in interested_visitors:
        # Τυχαία επιλογή ενός τύπου εισιτηρίου
        ticket_type = random.choice(event_ticket_types)

        buyer_has_ticket.append({
            "Buyer_Visitor_id": visitor_id,
            "Ticket_Ticket_id": None,  # Δεν έχουμε συγκεκριμένο εισιτήριο
            "Event_Event_id": event_id,
            "Ticket_type_Type": ticket_type,
            "datetime": datetime.now() - timedelta(days=random.randint(1, 30))
        })

# Αποθήκευση
pd.DataFrame(buyer_has_ticket).to_csv("generated/buyer_has_ticket.csv", index=False)

<ipython-input-59-cdc37f7dbf28>:3: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  ticket_df = pd.read_csv("generated/ticket.csv")


In [60]:
#resale_queue
# Join ticket με event για να έχουμε την ημερομηνία του event
ticket_with_event = ticket_df.merge(
    event_df[["Event_id", "Date"]],
    left_on="Event_Event_id",
    right_on="Event_id"
)

# Φιλτράρουμε μόνο εισιτήρια:
# - που έχουν Seller (δηλαδή μπήκαν για resale)
# - και το event τους είναι μελλοντικό
ticket_on_resale = ticket_with_event[
    ticket_with_event["Seller_Seller_id"].notna() &
    (pd.to_datetime(ticket_with_event["Date"]) > datetime.now())
]

# Δημιουργούμε ResaleQueue
resale_queue = {
    "Ticket_Ticket_id": [],
    "TimestampDATETIME": []
}

for _, row in ticket_on_resale.iterrows():
    resale_queue["Ticket_Ticket_id"].append(row["Ticket_id"])
    resale_queue["TimestampDATETIME"].append(
        datetime.now() - timedelta(days=random.randint(1, 10))  # Ημερομηνία δήλωσης μεταπώλησης
    )

# Αποθήκευση
pd.DataFrame(resale_queue).to_csv("generated/resalequeue.csv", index=False)


In [61]:
#ratings
os.makedirs("generated", exist_ok=True)

# Χάρτης event_id -> ημερομηνία
event_dates = pd.to_datetime(event_df.set_index("Event_id")["Date"]).to_dict()

# Χάρτης event -> performances
event_to_performances = performance_df.groupby("Event_Event_id")["Performance_id"].apply(list).to_dict()

ratings = {
    "Rating_id": [],
    "ArtistPerformanceINT": [],
    "SoundAndLightingINT": [],
    "StagePresenceINT": [],
    "OrganizationINT": [],
    "OverallImpressionINT": [],
    "Performance_Performance_id": [],
    "Visitor_Visitor_id": []
}

rating_id = 1

# Για κάθε ticket → ψάχνουμε valid events που έχουν περάσει
for _, row in ticket_df.iterrows():
    visitor_id = row["Visitor_Visitor_id"]
    event_id = row["Event_Event_id"]

    # Ελέγχουμε αν υπάρχει η ημερομηνία event και αν έχει ήδη πραγματοποιηθεί
    event_date = event_dates.get(event_id)
    if not event_date or event_date > datetime.now():
        continue  # Το event δεν έχει γίνει ακόμα

    # Παίρνουμε τις performances του event
    possible_performances = event_to_performances.get(event_id, [])
    if not possible_performances:
        continue

    # Επιλέγουμε τυχαία 1–3 performances για αξιολόγηση
    num_to_rate = random.randint(1, min(3, len(possible_performances)))
    rated_performances = random.sample(possible_performances, num_to_rate)

    for perf_id in rated_performances:
        ratings["Rating_id"].append(rating_id)
        ratings["ArtistPerformanceINT"].append(random.randint(1, 5))
        ratings["SoundAndLightingINT"].append(random.randint(1, 5))
        ratings["StagePresenceINT"].append(random.randint(1, 5))
        ratings["OrganizationINT"].append(random.randint(1, 5))
        ratings["OverallImpressionINT"].append(random.randint(1, 5))
        ratings["Performance_Performance_id"].append(perf_id)
        ratings["Visitor_Visitor_id"].append(visitor_id)
        rating_id += 1

# Αποθήκευση
pd.DataFrame(ratings).to_csv("generated/rating.csv", index=False)


NameError: name 'performance_df' is not defined